# Import Libraries 

In [38]:
import pandas as pd
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy import units as u
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Login to Gaia to access data

In [4]:
# Set the environment variable within the notebook 
os.environ['SSL_CERT_FILE'] = '/usr/local/ciao-4.13/ciao-4.13/ots/lib/python3.7/site-packages/certifi/cacert.pem'

In [32]:
Gaia.login()

INFO: Login to gaia TAP server [astroquery.gaia.core]
User: jperea
Password: ········
INFO: OK [astroquery.utils.tap.core]
INFO: Login to gaia data server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]


# Read in X-ray file path and create data frame
This file is the X-ray catalog of sources in IC 342 that we found at a probability of 99%. 

142 Sources. 
This only includes the X-ray sources that are within our HST observation. 

The columns are source name, ra, dec, -, -, sig1, sig2, sig3. 
sig1,2, and 3 refer to the positional uncertainty of Chandra at 1, 2, and 3 sigma confidence intervals. 

In [6]:
file_path_Xray = 'Xray99_final.txt'

In [7]:
df_Xray = pd.read_csv(file_path_Xray, sep=',', header=None)

### Change column names 

In [8]:
column_names_Xray = ['source', 'ra', 'dec', '?', '?', 'sig1', 'sig2', 'sig3']

In [9]:
df_Xray.columns = column_names_Xray

In [10]:
# dropping two of the columns we do not need
drop_col_Xray = [col for col in df_Xray.columns if '?' in col]

df_Xray = df_Xray.drop(columns=drop_col_Xray)

In [11]:
df_Xray

,source,ra,dec,sig1,sig2,sig3
0,034642.35+680938.2,56.676577,68.160591,2.561205,5.122410,7.683615
1,034643.19+680211.6,56.680053,68.036526,2.471818,4.943636,7.415454
2,034702.73+681001.6,56.761489,68.167083,2.316915,4.633831,6.950746
3,034644.91+680859.1,56.687242,68.149708,2.210881,4.421761,6.632642
4,034619.81+680837.5,56.582643,68.143720,2.182250,4.364499,6.546749
...,...,...,...,...,...,...
137,034652.81+680504.5,56.720169,68.084545,0.493928,0.987855,1.481783
138,034559.57+680538.0,56.498318,68.093858,0.493555,0.987110,1.480664
139,034648.44+680547.2,56.701931,68.096418,0.483667,0.967335,1.451002
140,034643.65+680611.3,56.681968,68.103121,0.482793,0.965587,1.448380


# Query Gaia

We want to compare our positions in the X-ray catalog to positions in the Gaia database to see if we have any foreground stars in our X-ray list. 

In [33]:
def query_gaia_adql(ra, dec, radius):
    """
    Queries the Gaia catalog for sources around a given position (RA, Dec) within a specified radius using ADQL.

    Parameters:
    ra (float): Right Ascension in degrees.
    dec (float): Declination in degrees.
    radius (float): Search radius in arcseconds.

    Returns:
    astropy.table.Table: Results from the Gaia query.
    """
    # Convert radius from arcseconds to degrees
    radius_deg = radius / 3600.0

    # Construct ADQL query
    query = f"""
    SELECT * FROM gaiadr3.gaia_source
    WHERE DISTANCE(POINT({ra}, {dec}), POINT(ra, dec)) < {radius_deg}
    """
    
    # Perform the query using Astroquery's Gaia module
    job = Gaia.launch_job_async(query)
    result = job.get_results()
    
    # Return the result of the query
    return result

In [34]:
# Initialize a list to hold the results from Gaia
all_results = []

# Loop through each row in the DataFrame
for index, row in df_Xray.iterrows():
    # Extract the RA, Dec, and search radius (sig3) for the current row
    source_id = row['source']
    ra = row['ra']
    dec = row['dec']
    radius = row['sig3']
    
    # Query Gaia using the current row's parameters
    result = query_gaia_adql(ra, dec, radius)
    
    # Convert result to pandas DataFrame
    df_result = result.to_pandas()
    
    # Add a column to the DataFrame to indicate the source identifier
    df_result['query_source'] = source_id
    
    # Append the DataFrame to the list
    all_results.append(df_result)

INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


## Move all results into a data frame

In [35]:
# Concatenate all the individual DataFrames into one DataFrame
df_Gaia_results = pd.concat(all_results, ignore_index=True)


In [37]:
df_Gaia_results.columns

Index(['solution_id', 'DESIGNATION', 'SOURCE_ID', 'random_index', 'ref_epoch',
       'ra', 'ra_error', 'dec', 'dec_error', 'parallax',
       ...
       'azero_gspphot_lower', 'azero_gspphot_upper', 'ag_gspphot',
       'ag_gspphot_lower', 'ag_gspphot_upper', 'ebpminrp_gspphot',
       'ebpminrp_gspphot_lower', 'ebpminrp_gspphot_upper', 'libname_gspphot',
       'query_source'],
      dtype='object', length=153)

# Filter sources with parallax 

If Gaia is able to measure parallax then the source had to be measured within the Galaxy. 

In [47]:
# Filter sources with non-zero parallax
measurable_parallax_df = df_Gaia_results[df_Gaia_results['parallax'] > 0]

# Display the first few rows of the filtered DataFrame
measurable_parallax_df

,solution_id,DESIGNATION,SOURCE_ID,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,...,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot,query_source
0,1636148068921376768,Gaia DR3 493584311742730240,493584311742730240,638876494,2016.0,56.675081,0.308686,68.159966,0.389643,0.974275,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,034642.35+680938.2
2,1636148068921376768,Gaia DR3 493477762194779392,493477762194779392,903113288,2016.0,56.602521,0.870764,68.052598,1.004939,3.915352,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,034624.46+680312.1
3,1636148068921376768,Gaia DR3 493583968146334592,493583968146334592,662806445,2016.0,56.744889,0.251914,68.130734,0.335558,0.400624,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,034658.97+680750.8
4,1636148068921376768,Gaia DR3 493476868839800320,493476868839800320,379356935,2016.0,56.765787,0.211691,68.015351,0.321089,1.944744,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,034703.68+680054.2
5,1636148068921376768,Gaia DR3 493476868842805248,493476868842805248,664253801,2016.0,56.767141,0.089059,68.015481,0.119269,0.161899,...,0.0220,0.1381,0.0502,0.0166,0.1053,0.0271,0.0089,0.0569,PHOENIX,034703.68+680054.2
7,1636148068921376768,Gaia DR3 493477074998225920,493477074998225920,1780322294,2016.0,56.749576,0.121432,68.027043,0.160114,0.194044,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,034700.47+680137.6
8,1636148068921376768,Gaia DR3 493489959900068352,493489959900068352,1149306553,2016.0,56.615958,0.210306,68.093034,0.287018,0.519766,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,034627.53+680534.9
11,1636148068921376768,Gaia DR3 493571147666402688,493571147666402688,1029617146,2016.0,56.855150,0.116982,68.085152,0.146372,0.119269,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,034725.31+680507.2
12,1636148068921376768,Gaia DR3 493477212441334400,493477212441334400,1494155621,2016.0,56.804365,0.031583,68.042667,0.039986,0.332536,...,1.2726,1.3805,1.0664,1.0249,1.1169,0.5889,0.5661,0.6167,MARCS,034713.01+680236.0
13,1636148068921376768,Gaia DR3 493571903580653952,493571903580653952,1035605676,2016.0,56.826542,0.056239,68.095643,0.071828,0.622998,...,2.0507,2.2192,1.5664,1.5224,1.6574,0.8598,0.8354,0.9092,MARCS,034718.52+680544.8
